# 데이터 증강

In [27]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

## 하이퍼 파라미터

In [13]:
EPOCHS = 100
IMG_SIZE = 224

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',padding='SAME', input_shape=(IMG_SIZE,IMG_SIZE,3));
        self.pool1 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu',padding='SAME');
        self.pool2 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu',padding='SAME');
        self.pool3 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv4 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu',padding='SAME');
        self.pool4 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv5 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu',padding='SAME');
        self.pool5 = tf.keras.layers.MaxPool2D(padding='SAME')
        
        self.flat = tf.keras.layers.Flatten()
        self.den1 = tf.keras.layers.Dense(1024, activation='relu')
        self.drop = tf.keras.layers.Dropout(0.4)
        self.den2 = tf.keras.layers.Dense(1,activation='sigmoid')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.conv4(x)
        x = self.pool4(x)
        x = self.conv5(x)
        x = self.pool5(x)
        
        x = self.flat(x)
        x = self.den1(x)
        x = self.drop(x)
        return self.den2(x)
        

## 데이터셋 준비

In [6]:
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir,'data')
camera_dir = os.path.join(data_dir,'crawled_images')
camera0_dir = os.path.join(camera_dir,'0')
camera1_dir = os.path.join(camera_dir,'1')

normal_img_files = os.listdir(camera0_dir)
fire_img_files = os.listdir(camera1_dir)

In [9]:
for img in normal_img_files:
    img_path = os.path.join(camera0_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)

for img in fire_img_files:
    img_path = os.path.join(camera1_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)
        
normal_img_files = os.listdir(camera0_dir)
fire_img_files = os.listdir(camera1_dir)

img92.jpg P
(168, 301)


In [11]:
print(len(normal_img_files))
print(len(fire_img_files))

83
88


In [53]:
N_N_TRAIN = 3 * (len(normal_img_files) // 4)
train_x = []
train_y = []
test_x = []
test_y = []
for i, file in enumerate(normal_img_files):
    img_path = os.path.join(camera0_dir, file)
    image = Image.open(img_path)
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image = np.array(image).astype(np.float32)
    if i < N_N_TRAIN:
        train_x.append(image)
        train_y.append(0)
    else:
        test_x.append(image)
        test_y.append(0)

In [54]:
N_F_TRAIN = 3 * (len(fire_img_files) // 4)
for i, file in enumerate(fire_img_files):
    img_path = os.path.join(camera1_dir,file)
    image = Image.open(img_path)
    image = image.resize((IMG_SIZE,IMG_SIZE))
    image = np.array(image).astype(np.float32)
    if i < N_F_TRAIN:
        train_x.append(image)
        train_y.append(1)
    else:
        test_x.append(image)
        test_y.append(1)

In [55]:
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

train_x, test_x = train_x / 255.0, test_x/255.0

In [56]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x,train_y)).shuffle(1000).batch(16).prefetch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(16).prefetch(32)

In [57]:
model = MyModel()
model.compile(optimizer = 'adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
8/8 [==============================] - 16s 2s/step - loss: 0.8920 - precision: 0.5343 - recall: 0.7515 - val_loss: 0.3828 - val_precision: 0.8462 - val_recall: 1.0000
Epoch 2/100
8/8 [==============================] - 11s 1s/step - loss: 0.2406 - precision: 0.9076 - recall: 0.9890 - val_loss: 0.2170 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 3/100
8/8 [==============================] - 11s 1s/step - loss: 0.6305 - precision: 0.9480 - recall: 0.9838 - val_loss: 0.3681 - val_precision: 1.0000 - val_recall: 0.9091
Epoch 4/100
8/8 [==============================] - 11s 1s/step - loss: 0.0238 - precision: 0.9926 - recall: 0.9866 - val_loss: 0.1374 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 5/100
8/8 [==============================] - 11s 1s/step - loss: 0.0328 - precision: 0.9868 - recall: 0.9966 - val_loss: 0.2755 - val_precision: 1.0000 - val_recall: 0.8636
Epoch 6/100
8/8 [==============================] - 11s 1s/step - loss: 0.0732 - precision: 0.9788 - recal

Epoch 46/100
8/8 [==============================] - 12s 2s/step - loss: 1.7441e-05 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5781 - val_precision: 1.0000 - val_recall: 0.8636
Epoch 47/100
8/8 [==============================] - 13s 2s/step - loss: 2.4591e-05 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4173 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 48/100
8/8 [==============================] - 16s 2s/step - loss: 7.7494e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.3635 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 49/100
8/8 [==============================] - 12s 2s/step - loss: 5.7350e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.3674 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 50/100
8/8 [==============================] - 12s 1s/step - loss: 2.1328e-05 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.3734 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 51/100
8/8 [==============================] - 12s 2s/step - loss: 1.3704e-

8/8 [==============================] - 11s 1s/step - loss: 1.7055e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4370 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 91/100
8/8 [==============================] - 11s 1s/step - loss: 1.1736e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4375 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 92/100
8/8 [==============================] - 11s 1s/step - loss: 2.9462e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4382 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 93/100
8/8 [==============================] - 11s 1s/step - loss: 2.7604e-06 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4389 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 94/100
8/8 [==============================] - 11s 1s/step - loss: 7.8859e-07 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4395 - val_precision: 1.0000 - val_recall: 0.9545
Epoch 95/100
8/8 [==============================] - 11s 1s/step - loss: 9.4952e-07 - precisio